In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from IPython.display import display, Markdown

import pandas as pd
import numpy as np

import json

np.__version__

/var/folders/b7/c67wpzpn25q0h072y9mmz_th0000gn/T/ipykernel_9640/2341679603.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


'1.26.3'

# Pull in data

In [2]:
PREV_PERFORMER_SPLIT_FILE = './data/22.3-OUTPUT-gpt4-performer-to-performer-list-str-dicitonary.csv'
unmerged_df = pd.read_csv(PREV_PERFORMER_SPLIT_FILE)
unmerged_df

,performer,performers_list_str
0,10cc,10cc|
1,ABBA,ABBA|
2,AWB,AWB|
3,Aaliyah,Aaliyah|
4,Aaron Neville,Aaron Neville|
...,...,...
2457,Michael Jackson & Justin Timberlake,Michael Jackson|Justin Timberlake|
2458,Owl City,Owl City|
2459,"Pitbull Featuring Ne-Yo, Afrojack & Nayer",Pitbull|Ne-Yo|Afrojack|Nayer|
2460,Soko,Soko|


# General approach:

It won't necessarily be perfect. 

However, high accuracy pretty much only matters for *very popular* performers.
1. We'll likely run into them as we test the project (we have)
2. The main purpose is to be more accurate when comparing, say, Beyonce vs GaGa. If there is a Joe Shmmo and a Joseph Schmo, but he is a random with 2 songs, not merging him will likely not even be relevent to the big artists we'll reference agg stats for.

### 1. Apply manually identified merges 
This could probably be a gSheet that anyone can add to, and I can just read from here.

### 2. Low hanging fruit: finding mismatching case & small Levenshtien distances

### 3. OPTIONAL: now that we've done most of what we can, try embedding & visualizing in 2D (maybe Nomic makes this easy?)

We could scan visually in the viz.

But we could also programmatically pull up in a table any pairs who are close to each other.

# 1. Manually identified merges

In [3]:
# Pull from gSheet
G_SHEET_URL = "https://docs.google.com/spreadsheets/d/1RUeuDF6yhv8xnoPjQoNa9hXsYDfNQzuOaWkSyww6Vi8/export?format=csv&gid=0"
g_sheet_manual_merges_df = pd.read_csv(G_SHEET_URL)
g_sheet_manual_merges_df

,final_name_to_use,name_1,name_2,name_3,name_4
0,Jay-Z,JAY-Z,JAY Z,Jay-Z,Jay Z
1,Beyonce,Beyonce Knowles,Beyonce,NaN,NaN
2,Stevie Wonder,Stevie Wonder,Little Stevie Wonder,NaN,NaN
3,Miley Cyrus,Hannah Montana,Miley Cyrus,NaN,NaN
4,Wizkid,Wizkid,WizKid,NaN,NaN
5,Eve,Eve,EVE,NaN,NaN
6,Nas,Nas,NAS,NaN,NaN
7,Soulja Boy,Soulja Boy Tell 'em,Soulja Boy Tell'em,NaN,NaN
8,Dion DiMucci,Dion Di Muci,Dion (Di Muci),NaN,NaN
9,Lil' Kim,Lil Kim,Lil' Kim,NaN,NaN


In [4]:
# Create a copy of unmerged_df
merged_df = unmerged_df.copy()

# DROP MISC
# Replace any sub strings reading "his Orchestra|" or "His Orchestra|" with ""
merged_df['performers_list_str'] = merged_df['performers_list_str'].str.replace("his Orchestra|", "", regex=False)
merged_df['performers_list_str'] = merged_df['performers_list_str'].str.replace("His Orchestra|", "", regex=False)

# Iterate over each row in g_sheet_manual_merges_df
for index, row in g_sheet_manual_merges_df.iterrows():
    for col in ['name_1', 'name_2', 'name_3', 'name_4']:
        name_to_replace = row[col]
        if pd.notna(name_to_replace):
            # Find names to replace by searching for them as substrings of performers_list_str
            merged_df['performers_list_str'] = merged_df['performers_list_str'].str.replace(f"{name_to_replace}|", f"{row['final_name_to_use']}|", regex=False)


# Check that it worked:
for index, row in g_sheet_manual_merges_df.iterrows():
    for col in ['name_1', 'name_2', 'name_3', 'name_4']:
        name_to_replace = row[col]
        if pd.notna(name_to_replace) and name_to_replace != row['final_name_to_use']:
            assert(merged_df['performers_list_str'].str.contains(f"{name_to_replace}|", regex=False).sum() == 0)



# 2. Verify the Low hanging fruit: mismatched case and Levenshtein distance

## Verify that there are no overlooked merges based purely on case, eg "Nas" and "NAS"

In [5]:
# Parse using "|" the performers_list_str back into a list, and combine them into one big, array of unique values
performers_list = merged_df['performers_list_str'].str.split("|").apply(lambda x: [i for i in x if i]).values
performers_list = np.concatenate(performers_list)
performers_list = np.unique(performers_list)

In [6]:
# Now lower case all the names, and see if the length changes
performers_list_lower = np.char.lower(performers_list)

# VERIFY: Now, print out any names that appear more than once
duplicates = pd.Series(performers_list_lower).value_counts()
duplicates = duplicates[duplicates > 1]
assert(len(duplicates) == 0)

## Check for almost identical names

In [7]:
from Levenshtein import distance

# Now, compare the distance(performer1, performer2) between every single performer
# Filter to only those that are within a certain distance
MIN_DISTANCE = 2
performer_distances = []
for i, performer1 in enumerate(performers_list):
    for j, performer2 in enumerate(performers_list):
        if i < j:
            if distance(performer1, performer2) <= MIN_DISTANCE:
                performer_distances.append((performer1, performer2))

# Now, print out the performer_distances
performer_distances


[('112', '702'),
 ('112', 'D12'),
 ('112', 'U2'),
 ('4PM', 'M'),
 ('702', 'D12'),
 ('702', 'U2'),
 ('?', 'Do'),
 ('?', 'M'),
 ('?', 'MO'),
 ('?', 'U2'),
 ('?', 'YG'),
 ('ABBA', 'ABC'),
 ('ABBA', 'AWB'),
 ('ABBA', 'BBD'),
 ('ABC', 'AJR'),
 ('ABC', 'AWB'),
 ('ABC', 'Ace'),
 ('ABC', 'Ali'),
 ('ABC', 'BBD'),
 ('ABC', 'TLC'),
 ('AJR', 'AWB'),
 ('AJR', 'Ace'),
 ('AJR', 'Ali'),
 ('AWB', 'Ace'),
 ('AWB', 'Ali'),
 ('AWB', 'SWV'),
 ('Ace', 'Ali'),
 ('Ace', 'Eve'),
 ('Ace', 'Joe'),
 ('Akon', 'Dion'),
 ('Akon', 'Loon'),
 ('Al Wilson', 'Ann Wilson'),
 ('Ali', 'Alias'),
 ('Ali', 'Asia'),
 ('Alias', 'Asia'),
 ('Alias', 'Plies'),
 ('America', 'Amerie'),
 ('Aqua', 'Asia'),
 ('Asia', 'Sia'),
 ('B2K', 'BBD'),
 ('B2K', 'BTS'),
 ('B2K', 'Bob'),
 ('B2K', 'U2'),
 ('BBD', 'BTS'),
 ('BBD', 'Bob'),
 ('BBD', 'JID'),
 ('BBD', 'LTD'),
 ('BTS', 'Bob'),
 ('BTS', 'DRS'),
 ('BTS', 'LTD'),
 ('Beck', 'Brick'),
 ('Bob', 'Do'),
 ('Bob', 'Hot'),
 ('Bob', 'Joe'),
 ('Bob', 'Lobo'),
 ('Bobby Bare', 'Bobby Hart'),
 ('Bread', '

In [8]:
# Now filter performer_distances to just names that are more than 3 letters
performer_distances_filtered = []
MIN_LENGTH = 5
for performer1, performer2 in performer_distances:
    if len(performer1) > MIN_LENGTH and len(performer2) > MIN_LENGTH:
        performer_distances_filtered.append((performer1, performer2))

performer_distances_filtered

[('Al Wilson', 'Ann Wilson'),
 ('America', 'Amerie'),
 ('Bobby Bare', 'Bobby Hart'),
 ('Cassidy', 'Cassie'),
 ('Cat Stevens', 'Ray Stevens'),
 ('Dan Hill', 'Dru Hill'),
 ('Firehouse', 'Lifehouse'),
 ('Freddie Cannon', 'Freddy Cannon'),
 ('H-Town', 'O-Town'),
 ('Jim Jones', 'Jimmy Jones'),
 ('Jim Jones', 'Joe Jones'),
 ('Jim Jones', 'Tom Jones'),
 ('Jimmy Page', 'Tommy Page'),
 ('Joe Jonas', 'Joe Jones'),
 ('Joe Jones', 'Tom Jones'),
 ('Johnny Cash', 'Johnny Nash'),
 ('King Floyd', 'Pink Floyd'),
 ('Monica', 'Monifah'),
 ('Nelson', 'Nilsson'),
 ('Oliver', 'Olivia'),
 ('Rick Nelson', 'Ricky Nelson'),
 ('Robert Palmer', 'Robert Parker'),
 ('Sam Smith', 'Sammi Smith'),
 ('The Beaters', 'The Beatles'),
 ('The Bells', 'The Dells'),
 ('The Bells', 'The Drells'),
 ('The Capris', 'The Cars'),
 ('The Cars', 'The Cure'),
 ('The Cars', 'The Larks'),
 ('The Cars', 'The Tams'),
 ('The Deele', 'The Dells'),
 ('The Dells', 'The Dovells'),
 ('The Dells', 'The Drells'),
 ('The Dovells', 'The Drells'),
 

# Ok, looks pretty good, export time!

In [9]:
PERFORMER_SPLIT_FILE = "./data/22.4-OUTPUT-performer_list_str-with-multi-name-artists-merged.csv"
merged_df.to_csv('./data/22.4-OUTPUT-performer_list_str-with-multi-name-artists-merged.csv', index=False)